# Simplest Setup

In [ ]:
import concurrent.futures

def background_function(arg1, arg2):
    # Your function implementation here
    return arg1 + arg2

executor = concurrent.futures.ThreadPoolExecutor()
future = executor.submit(background_function, 2, 5)

# To get the result later
result = future.result()
result

7

# More Flexible/Advanced Setups

In [1]:
import time
from concurrent.futures import ThreadPoolExecutor
from IPython.display import display, Javascript

def long_running_function(seconds):
    print(f"Starting a long-running task that will take {seconds} seconds...")
    time.sleep(seconds)
    return f"Task finished after {seconds} seconds."

def done_callback(future):
    result = future.result()
    print(result)
    # Notify user that the task is done by printing a message.
    display(Javascript('alert("Background task is complete!")')) # used to give the headsup in the Jupyter notebook

# Create a ThreadPoolExecutor to run the function in a separate thread
executor = ThreadPoolExecutor(max_workers=4)

# Starts the task and assigns a callback function that will be called upon completion
future = executor.submit(long_running_function, 3)
future.add_done_callback(done_callback)

future1 = executor.submit(long_running_function, 2)
future1.add_done_callback(done_callback)

future2 = executor.submit(long_running_function, 1)
future2.add_done_callback(done_callback)


# In Jupyter notebook, how can I run a python function in the background (non-blocking, so I can continue to work in the notebook and write/run other cells) and then be notified when the result is ready?


Starting a long-running task that will take 3 seconds...
Starting a long-running task that will take 2 seconds...
Starting a long-running task that will take 1 seconds...


In [2]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define the function you want to run in background threads
def long_running_task(task_id, duration):
    print(f"Task {task_id}: Starting a long task that takes {duration} seconds...")
    time.sleep(duration)
    return f"Task {task_id}: Finished after {duration} seconds."

# Number of tasks you want to run
number_of_tasks = 5

# Create a ThreadPoolExecutor with the desired number of workers
with ThreadPoolExecutor(max_workers=3) as executor:
    # A list to hold the future objects returned by submit()
    futures = []

    # Iterate and submit tasks with varying durations
    for i in range(number_of_tasks):
        duration = 2 + i # Example varying duration for demonstration
        future = executor.submit(long_running_task, i, duration)
        futures.append(future)

    # Process the results as they complete
    for future in as_completed(futures):
        result = future.result()
        print(result)

print("All tasks are complete.")

Task 0: Starting a long task that takes 2 seconds...
Task 1: Starting a long task that takes 3 seconds...
Task 2: Starting a long task that takes 4 seconds...


Task finished after 1 seconds.
Task finished after 2 seconds.
Task finished after 3 seconds.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Task 3: Starting a long task that takes 5 seconds...Task 0: Finished after 2 seconds.



<IPython.core.display.Javascript object>

Task 4: Starting a long task that takes 6 seconds...Task 1: Finished after 3 seconds.

Task 2: Finished after 4 seconds.
Task 3: Finished after 5 seconds.
Task 4: Finished after 6 seconds.
All tasks are complete.


# Delaying KeyboardInterrrupts until saving is done

In [6]:
import signal
import time

# Define a class to toggle the interrupt handling status
class DelayedKeyboardInterrupt:
    def __enter__(self):
        # Save the current signal handler for SIGINT
        self.signal_received = False
        self.old_handler = signal.signal(signal.SIGINT, self.handler)

    def handler(self, sig, frame):
        # Set a flag to indicate an interrupt was received
        # Directly print the message in the handler
        print('KeyboardInterrupt signal received. Program will exit as soon as saving is done.', flush=True)
        self.signal_received = (sig, frame)

    def __exit__(self, type, value, traceback):
        # Restore the original signal handler for SIGINT
        signal.signal(signal.SIGINT, self.old_handler)
        if self.signal_received:
            # If an interrupt was received, re-raise it now that it's safe
            self.old_handler(*self.signal_received)

# Your code where you want protection from interrupts
try:
    with DelayedKeyboardInterrupt():
        # Your critical section that should not be interrupted
        print("Starting critical section where saving to file occurs.")
        time.sleep(20)  # Simulate long-running save operation
        print("Finished critical section.")

    # It's now safe to handle the interrupt if one was received
    print("It's now safe to be interrupted.")

except KeyboardInterrupt:
    print("Program exiting after saving.")

# Rest of your code
print("Normal execution continues...")

Starting critical section where saving to file occurs.
KeyboardInterrupt signal received. Program will exit as soon as saving is done.
Finished critical section.
Program exiting after saving.
Normal execution continues...


In [ ]:
# Ignore keyboard interrupt
original_handler = signal.signal(signal.SIGINT, signal.SIG_IGN)

try:
    with open(filename, 'w') as file:
        file.write(data)
        # Simulate time-consuming saving process
        time.sleep(5)
finally:
    # Restore original handler
    signal.signal(signal.SIGINT, original_handler)